- **Name:** 14.2_sql_windfunc_ranking
- **Author:** Shamas Imran
- **Desciption:** Ranking functions in SQL using window functions
- **Date:** 19-Aug-2025
<!--
REVISION HISTORY
Version          Date        Author           Desciption
01           19-Aug-2025   Shamas Imran       Applied row_number, rank, dense_rank in SQL  
                                              Used PARTITION BY and ORDER BY  
                                              Validated ranking outputs with datasets  
-->

In [0]:
from pyspark.sql.types import *
from pyspark.sql import Row
from datetime import date
import random

# -------------------------------
# 1. Student DataFrame
# -------------------------------
student_schema = StructType([
    StructField('StudentID', IntegerType(), False),
    StructField('StudentName', StringType(), True),
    StructField('StudentAge', IntegerType(), True)
])

student_data = [
    (1, "Alice", 34), 
    (2, "Bob", 45), 
    (3, "Charlie", 29),
    (4, "Shamas", 40)
]

df_student = spark.createDataFrame(student_data, student_schema)

# -------------------------------
# 2. Course DataFrame
# -------------------------------
course_schema = StructType([
    StructField('CourseID', IntegerType(), False),
    StructField('CourseName', StringType(), True),
    StructField('CourseTitle', StringType(), True),
])

course_data = [
    (1, "Physics", "1111"), 
    (2, "Chemistry", "2222"), 
    (3, "English", "3333"),
    (4, "Computer Science", "4444")
]

df_course = spark.createDataFrame(course_data, course_schema)

# -------------------------------
# 3. Enrollment DataFrame
# -------------------------------
enrollment_schema = StructType([
    StructField("EnrollmentID", IntegerType(), False),
    StructField("StudentID_FK", IntegerType(), False),
    StructField("CourseID_FK", IntegerType(), False),
    StructField("EnrollmentDate", DateType(), True)
])

enrollment_data = [
    (1, 1, 1, date(2023, 9, 1)),   # Alice -> Physics
    (2, 2, 2, date(2023, 9, 2)),   # Bob -> Chemistry
    (3, 4, 4, date(2023, 9, 4)),   # Shamas -> Computer Science
    (4, 1, 2, date(2023, 9, 5)),   # Alice -> Chemistry
]

df_enrollment = spark.createDataFrame(enrollment_data, enrollment_schema)

# -------------------------------
# 4. Score DataFrame
# -------------------------------
semesters = ["2023-Spring", "2023-Fall", "2024-Spring", "2024-Fall", "2025-Spring"]
score_data = []
score_id = 1
enrollment_ids = [row.EnrollmentID for row in df_enrollment.collect()]

for enrollment_id in enrollment_ids:
    selected_semesters = random.sample(semesters, k=random.randint(2, 4))
    for sem in selected_semesters:
        score_data.append(Row(
            ScoreID=score_id,
            EnrollmentID_FK=enrollment_id,
            Semester=sem,
            Score=random.randint(60, 100)
        ))
        score_id += 1

score_schema = StructType([
    StructField("ScoreID", IntegerType(), False),
    StructField("EnrollmentID_FK", IntegerType(), False),
    StructField("Semester", StringType(), True),
    StructField("Score", IntegerType(), True)
])

df_score = spark.createDataFrame(score_data, schema=score_schema)

In [0]:
df_student.createOrReplaceTempView("Student")
df_course.createOrReplaceTempView("Course")
df_enrollment.createOrReplaceTempView("Enrollment")
df_score.createOrReplaceTempView("Score")

In [0]:
%sql
-- # 1. Rank students within each course by score
SELECT 
    st.StudentName,
    e.CourseID_FK,
    s.Score,
    RANK() OVER (PARTITION BY e.CourseID_FK ORDER BY s.Score DESC) AS Rank
FROM Score s
JOIN Enrollment e ON s.EnrollmentID_FK = e.EnrollmentID
JOIN Student st ON e.StudentID_FK = st.StudentID;

In [0]:
%sql
-- 1. Rank students within each course by score
SELECT 
    st.StudentName,
    e.CourseID_FK,
    s.Score,
    ROW_NUMBER() OVER (PARTITION BY e.CourseID_FK ORDER BY s.Score DESC) AS RowNum
FROM Score s
JOIN Enrollment e ON s.EnrollmentID_FK = e.EnrollmentID
JOIN Student st ON e.StudentID_FK = st.StudentID;

In [0]:
%sql
-- 2. Dense Rank (no gaps in ranking)
SELECT
    st.StudentName,
    e.CourseID_FK,
    s.Score,
    DENSE_RANK() OVER (
        PARTITION BY e.CourseID_FK     -- partition by course
        ORDER BY s.Score DESC          -- order scores in descending order
    ) AS DenseRank
FROM Score s
JOIN Enrollment e 
    ON s.EnrollmentID_FK = e.EnrollmentID
JOIN Student st 
    ON e.StudentID_FK = st.StudentID;


In [0]:
%sql
-- 3. Row number (unique sequence within partition)
SELECT
    st.StudentName,
    e.CourseID_FK,
    s.Score,
    ROW_NUMBER() OVER (
        PARTITION BY e.CourseID_FK     -- partition by course
        ORDER BY s.Score DESC          -- order scores in descending order
    ) AS RowNum
FROM Score s
JOIN Enrollment e 
    ON s.EnrollmentID_FK = e.EnrollmentID
JOIN Student st 
    ON e.StudentID_FK = st.StudentID;


In [0]:
%sql
-- 4. Top scorer per course (filtering rank = 1)
SELECT
    StudentName,
    CourseID_FK,
    Score,
    Rank
FROM (
    SELECT
        st.StudentName,
        e.CourseID_FK,
        s.Score,
        RANK() OVER (
            PARTITION BY e.CourseID_FK    -- partition by course
            ORDER BY s.Score DESC         -- order scores in descending order
        ) AS Rank
    FROM Score s
    JOIN Enrollment e 
        ON s.EnrollmentID_FK = e.EnrollmentID
    JOIN Student st 
        ON e.StudentID_FK = st.StudentID
) ranked
WHERE ranked.Rank = 1;   -- keep only top scorers
